<a href="https://colab.research.google.com/github/petroismRavi/logging-volvo/blob/main/Volve_logging_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import time

import warnings
warnings.filterwarnings("ignore")
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [7]:
#Loading data from volve data set
well_13 = pd.read_excel('https://raw.githubusercontent.com/petroismRavi/logging-volvo/main/data-logging-volvo.xlsx', sheet_name='well 13',index_col=0)
well_14 = pd.read_excel('https://raw.githubusercontent.com/petroismRavi/logging-volvo/main/data-logging-volvo.xlsx', sheet_name='well 14')
well_15 = pd.read_excel('https://raw.githubusercontent.com/petroismRavi/logging-volvo/main/data-logging-volvo.xlsx', sheet_name='well 15')

In [8]:
well_13.head()

,Well,GR,RT,RHOB,NPHI
Depth,,,,,
4175.5,13,20.6032,4.1812,2.6117,0.0770
4176.0,13,21.4990,4.5516,2.6131,0.0798
4176.5,13,22.4472,4.4804,2.6334,0.0801
4177.0,13,29.6713,4.3859,2.6328,0.1005
4177.5,13,34.7014,4.8566,2.6183,0.1001


In [9]:
well_14.head()

,Depth,Well,GR,RT,RHOB,NPHI,Facies
0,3178.5,14,50.2190,0.5888,2.3296,0.3657,SH
1,3179.0,14,47.2468,0.7768,2.3170,0.3776,UN
2,3179.5,14,49.5247,1.0707,2.2960,0.5390,SH
3,3180.0,14,44.9124,1.4460,2.2514,0.5482,UN
4,3180.5,14,47.0048,0.9542,2.2733,0.5076,UN


In [10]:
data=pd.concat([well_14, well_15],axis=0)
data['RT_log']=np.log10(data['RT'])

In [11]:
data.head()

,Depth,Well,GR,RT,RHOB,NPHI,Facies,RT_log
0,3178.5,14,50.2190,0.5888,2.3296,0.3657,SH,-0.230032
1,3179.0,14,47.2468,0.7768,2.3170,0.3776,UN,-0.109691
2,3179.5,14,49.5247,1.0707,2.2960,0.5390,SH,0.029668
3,3180.0,14,44.9124,1.4460,2.2514,0.5482,UN,0.160168
4,3180.5,14,47.0048,0.9542,2.2733,0.5076,UN,-0.020361


GR - Gamma Radioactivity

RT - Total Resistivity

RHOB - Density

NPHI - Neutron Porosity

Facies - Rock types and position

RT_Log - Resistivity of Uninvaded zone

In [ ]:
sns.pairplot(data.drop(['Well','Depth'], axis = 1) ,hue='Facies',diag_kind='hist')

In [ ]:
labels=[1,2,3,4]
data['Facies_labels']=np.select([data['Facies'] == 'SH',
                                data['Facies'] == 'UN',
                                data['Facies'] == 'SS',
                                data['Facies'] == 'CB',]
                                ,labels)

In [ ]:
data.head()

In [ ]:
facies_colors=['#2E86C1', '#196F3D','#F4D03F','#DC7633']
facies = ['SH', 'UN', 'SS', 'CB']

def log_plot(logs,facies_colors):
    logs=logs.sort_values(by='Depth')
    cmap_facies = colors.ListedColormap(
            facies_colors[0:len(facies_colors)], 'indexed')

    ztop=logs.Depth.min(); zbot=logs.Depth.max()

    cluster=np.repeat(np.expand_dims(logs['Facies_labels'].values,1), 100, 1)

    f,ax=plt.subplots(nrows=1,ncols=5,figsize=(8,12))
    ax[0].plot(logs.GR, logs.Depth, '-g')
    ax[1].plot(logs.RT_log, logs.Depth, '-')
    ax[2].plot(logs.NPHI, logs.Depth, '-', color='0.5')
    ax[3].plot(logs.RHOB, logs.Depth, '-', color='r')
    im=ax[4].imshow(cluster, interpolation='none', aspect='auto',
                   cmap=cmap_facies,vmin=1,vmax=4)

    divider = make_axes_locatable(ax[4])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar=plt.colorbar(im, cax=cax)
    cbar.set_label((50*' ').join(['SH', 'UN', 'SS', 'CB']))
    cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')

    for i in range(len(ax)-1):
        ax[i].set_ylim(ztop,zbot)
        ax[i].invert_yaxis()
        ax[i].grid()
        ax[i].locator_params(axis='x', nbins=3)

    ax[0].set_xlabel("GR")
    ax[0].set_xlim(logs.GR.min(),logs.GR.max())
    ax[1].set_xlabel("RT")
    ax[1].set_xlim(logs.RT_log.min(),logs.RT_log.max())
    ax[2].set_xlabel("NPHI")
    ax[2].set_xlim(logs.NPHI.min(),logs.NPHI.max())
    ax[3].set_xlabel("RHOB")
    ax[3].set_xlim(logs.RHOB.min(),logs.RHOB.max())
    ax[4].set_xlabel('Facies')

    ax[1].set_yticklabels([])
    ax[2].set_yticklabels([])
    ax[3].set_yticklabels([])
    ax[4].set_yticklabels([])
    ax[4].set_xticklabels([])
    f.suptitle('Well: %s'%logs.iloc[0]['Well'], fontsize=14,y=0.94)

In [ ]:
log_plot(data[data['Well'] == 14],facies_colors)

In [ ]:
#Bar chart: Distribution of Facies
Facies_dist = data['Facies'].value_counts().sort_index()

Facies_dist.plot(kind='bar',color=facies_colors,
                   title='Distribution of Facies')

In [ ]:
data.set_index('Depth',inplace=True)


In [ ]:
data.groupby("Well").count()

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
X = data.drop(['Facies','Well','Facies_labels','RT_log'], axis = 1) #Features: Feature vector
y = data['Facies']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import IsolationForest, RandomForestClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)
print('Dimensions of X_train:',X_train.shape)
print('Dimensions of X_test:',X_test.shape)

In [ ]:
X_train.hist()
plt.show()

### Scaling

In [ ]:
#Types of scaling methods  -
#1) Min Max Scaler
#2) Standard Scaler
#3) Max Abs Scaler
#4) Robust Scaler
#5) Quantile Transformer Scaler
#6) Power Transformer Scaler
#7) Unit Vector Scaler

In [ ]:
# Robust Scaler
rscaler=RobustScaler()#instantiate
rscaler.fit(X_train)

In [ ]:
X_train_scaled = rscaler.transform(X_train) # transform the train dataset to standardized data

# Original training dataset
print("Original median : %s " % rscaler.center_)
print("Original IQR : %s " % rscaler.scale_)

#Scaled training dataset
print("Scaled median : %s " % np.median(X_train_scaled,axis=0))
print("Scaled IQR : %s " % (np.percentile(X_train_scaled, 75,axis=0)-np.percentile(X_train_scaled, 25,axis=0)))

In [ ]:
#Scale the test data using the parameters learnt from the training dataset
X_test_scaled  = rscaler.transform(X_test)

print("Median of scaled test data: %s" % np.median(X_test_scaled,axis=0))
print("IQR of scaled test data: %s " % (np.percentile(X_test_scaled, 75,axis=0)-np.percentile(X_test_scaled, 25,axis=0)))

In [ ]:
pd.DataFrame(X_train_scaled,index=X_train.index, columns=X_test.columns).hist()
plt.show()

In [ ]:
X_train=pd.DataFrame(X_train_scaled,index=X_train.index, columns=X_test.columns)
X_test=pd.DataFrame(X_test_scaled,index=X_test.index, columns=X_test.columns)
X_test

### Outlier Detection - Isolation Forest

##### In principle, outliers are less frequent than regular observations and are different from them in terms of values (they lie further away from the regular observations in the feature space).

In [ ]:
# Isolation Forest
iforest = IsolationForest(n_estimators=200, contamination=0.5/100)
iforest = iforest.fit(X_train) #Training the model

In [ ]:
#Predictions
X_train_predict = iforest.predict(X_train)
X_train['Predict']=X_train_predict
X_train['Predict'] = X_train['Predict'].astype('category')
X_train

In [ ]:
#Visualization
sns.pairplot(X_train,hue='Predict',diag_kind='hist')

In [ ]:
#Removing Outliers
X_train['y_train']=y_train
X_train = X_train[X_train['Predict'] == 1]

y_train=X_train['y_train']
X_train = X_train.drop(['Predict','y_train'], axis = 1)

In [ ]:
print(len(X_train))
print(len(y_train))

In [ ]:
#Visualization Without Outliers
sns.pairplot(X_train,diag_kind='hist')

Logistic Regression
----

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

In [ ]:
print(logmodel.coef_)

In [ ]:
print(logmodel.intercept_)

In [ ]:
predictions=logmodel.predict(X_test)

In [ ]:
#Compare against true labels (Accuracy)
print('Accuracy (generalization)',logmodel.score(X_test,y_test)) #Accuracy (generalization)
print('Accuracy (memorization)',logmodel.score(X_train,y_train)) #Accuracy (memorization)

In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report, confusion_matrix

In [ ]:
#Comparing other metrics (f1_score)
print('F1_score (generalization)',f1_score(y_test,logmodel.predict(X_test),average="weighted")) #Accuracy (generalization)
print('F1_score (memorization)',f1_score(y_train,logmodel.predict(X_train),average="weighted")) #Accuracy (memorization)

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
#Confusion Matrix
names = ['SH', 'UN', 'SS', 'CB']

cf_matrix = confusion_matrix(y_train, logmodel.predict(X_train))
cf_matrix
cf=sns.heatmap(cf_matrix, annot=True, annot_kws={"size": 12},cmap='Blues',fmt="d",xticklabels=names,yticklabels=names)

plt.show()

Random Forrest  
---

In [ ]:
cforest=RandomForestClassifier(criterion='entropy',n_estimators=100,max_depth=5,random_state=1, n_jobs=2) #Creating Instance
cforest.fit(X_train, y_train) #Learning the decision boundaries
y_pred = cforest.predict(X_test)

In [ ]:
#Compare against true labels (Accuracy)
print('Accuracy (generalization)',cforest.score(X_test,y_test)) #Accuracy (generalization)
print('Accuracy (memorization)',cforest.score(X_train,y_train)) #Accuracy (memorization)

In [ ]:
#Comparing other metrics (f1_score)
print('F1_score (generalization)',f1_score(y_test,cforest.predict(X_test),average="weighted")) #F1_score (generalization)
print('F1_score (memorization)',f1_score(y_train,cforest.predict(X_train),average="weighted")) #F1_score (memorization)

In [ ]:
print(classification_report(y_test,y_pred))

Suppport Vector Machine
----

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC, SVC

#Defining parameter range
Tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                     'C': [0.1, 1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
                    {'kernel': ['poly'],'C': [1, 10, 100, 1000],'degree': [2,3,4]}]

svc_mod = GridSearchCV(SVC(), Tuned_parameters, refit = True, verbose = 3)

In [ ]:
svc_mod.fit(X_train, y_train)
svc_mod.best_estimator_

In [ ]:
print(svc_mod.best_params_)

In [ ]:
#Performance (Accuracy)
print('Accuracy (generalization)',svc_mod.score(X_test,y_test)) #Accuracy (generalization)
print('Accuracy (memorization)',svc_mod.score(X_train,y_train)) #Accuracy (memorization)

#Comparing other metrics (f1_score)
print('F1_score (generalization)',f1_score(y_test,svc_mod.predict(X_test),average="weighted")) #F1_score (generalization)
print('F1_score (memorization)',f1_score(y_train,svc_mod.predict(X_train),average="weighted")) #F1_score (memorization)

In [ ]:
srbf=SVC(C=100,kernel='rbf',gamma=0.1)
srbf.fit(X_train, y_train)

In [ ]:
#Performance (Accuracy)
print('Accuracy (generalization)',srbf.score(X_test,y_test)) #Accuracy (generalization)
print('Accuracy (memorization)',srbf.score(X_train,y_train)) #Accuracy (memorization)

#Comparing other metrics (f1_score)
print('F1_score (generalization)',f1_score(y_test,srbf.predict(X_test),average="weighted")) #F1_score (generalization)
print('F1_score (memorization)',f1_score(y_train,srbf.predict(X_train),average="weighted")) #F1_score (memorization)

In [ ]:
#Confusion Matrix
cf_matrix = confusion_matrix(y_train, srbf.predict(X_train))

sns.heatmap(cf_matrix, annot=True, annot_kws={"size": 12},cmap='Blues',fmt="d",xticklabels=names,yticklabels=names)

plt.show()